# In this notebook we are going to analyse the data scrapped from jumia.cm

In [1]:
# importing modules
import numpy as np
import pandas as pd
import re 
%matplotlib inline
import matplotlib.pyplot as plt
import time

In [2]:
# loading the scrapped data
df = pd.read_csv('./jumia_cm_smartphones_scraped_on_Sun_May_26_21:47:08_2019.csv')
# Getting somary statistic about our data
print(df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 910 entries, 0 to 909
Data columns (total 13 columns):
name                    910 non-null object
seller                  910 non-null object
price                   910 non-null float64
number-of-items-sold    863 non-null float64
ram                     796 non-null float64
rom                     824 non-null float64
screen-size             768 non-null float64
color                   910 non-null object
front-camera            797 non-null float64
rear-camera             797 non-null float64
number_of_sims          910 non-null int64
date                    910 non-null object
os                      872 non-null object
dtypes: float64(7), int64(1), object(5)
memory usage: 92.5+ KB
None


## Description of the data analysis
From the above result, we clearly see that our data have 910 rows. Also we clearly see that some columns entries are missing. we can see that we have the following: 
1. names has 910 entries so no missing data
2. seller has 910 entries so no missing data
3. price has 910 entries so no missing data
4. number-of-items-sold  has 863 so we have missing data here
5. ram has 796 so we have missing data here
6. rom has 824 so we have missing data here
7. screen-size has 768 so we have missing data here
8. color has 910 so no missing data here
9. front-camera has 797 so we have missing data here
10. rear-camera has 797 so we have missing data here
11. number-of-sim has 910 so we have  no missing data here
12. date has no missing data and it worth noting that we filled the date property after scrape data on the said item
13. os has 872 entries so we have missing data here.

The pandas info methode is really good if we want to identifies missing data from our dataset.

# CLEANING OUR DATASET
 We have a messy data that we should see how to clean it

In [9]:
# We can use pandas dropna method to derop all rows with NAN values
#droping rows with missing values
#without_missing_values = df.dropna()
#getting the description of the data without missing values
#without_missing_values.describe()
# print(without_missing_values['os'].astype('category').value_counts())
# pd.value_counts(df['os']=='ios')
# os = df['os'].astype('category').value_counts()
#price = df['price'].astype('category').value_counts()
#sold = df['number-of-items-sold'].astype('category').value_counts()
#print(sold)
#print(os)
#print(price)
# bins=['android', 'ios']
# #pd.cut(df, bins)
#below= (df['price']<50000).value_counts()
#print(below)
#print(df['number-of-items-sold ']).max()

In [3]:
''' Getting the summary of the dataset'''
summary = df.describe()
print(summary)

''' From the summary statistic we clearly see that the lower price is 5000xaf around 8.6usd wish is not right so lets
check if actually it is a phone. We will do this by subsetting our dataset even below 15000xaf(17.2usd)'''

least_item= df.loc[df['price']<=15000]
print(least_item)

'''We clearly see that  Blitzwolf BW-BTS1 Sport Bluetooth Earphone Head, SP4 Senior - 32Mo - 4Mo RAM - 2,2" - Blanc,
 Chargeur Pour Samsung Galaxy - Noir are not smart phones so we should drop them '''
df=df.drop([351, 612, 908], axis=0)

              price  number-of-items-sold         ram          rom  \
count  9.100000e+02            863.000000  796.000000   824.000000   
mean   1.814635e+05           1968.893395    2.919719    50.531553   
std    1.816174e+05           9288.460326    2.354279    92.792725   
min    5.000000e+03              9.000000    0.512000     4.000000   
25%    7.500000e+04             49.000000    1.000000    16.000000   
50%    1.339475e+05            101.000000    3.000000    32.000000   
75%    2.120000e+05           1001.000000    4.000000    64.000000   
max    2.210000e+06         100001.000000   32.000000  1616.000000   

       screen-size  front-camera  rear-camera  number_of_sims  
count   768.000000    797.000000   797.000000      910.000000  
mean      5.496094      5.196989    12.856462        1.501099  
std       0.746367      3.979358     5.041431        0.500274  
min       0.000000      0.000000     0.300000        1.000000  
25%       5.000000      2.000000    12.000000    

In [10]:
''' Lets drop the columns that has missing values so that we can have good data to work with
we can achieve this by using the dropna pandas method'''

smartphones = df.dropna()


''' Getting a new summary of the data'''
print('-----------------------------SUMMARY STATISTICS--------------------------------------------')
smartphones.describe()


# print('--------------------------CUMMULATIVE SUM-----------------------------------------------------')
# print(df[.cumsum())

-----------------------------SUMMARY STATISTICS--------------------------------------------


,price,number-of-items-sold,ram,rom,screen-size,front-camera,rear-camera,number_of_sims
count,7.660000e+02,766.000000,766.000000,766.000000,766.000000,766.000000,766.000000,766.000000
mean,1.815186e+05,2174.036554,2.923102,50.754569,5.495692,5.200783,12.878068,1.505222
std,1.882411e+05,9827.540757,2.381297,95.530099,0.747261,3.966115,5.065221,0.500299
min,1.750000e+04,9.000000,0.512000,4.000000,0.000000,0.000000,0.300000,1.000000
25%,7.250000e+04,99.000000,1.000000,16.000000,5.000000,2.000000,12.000000,1.000000
50%,1.295000e+05,101.000000,3.000000,32.000000,5.500000,5.000000,13.000000,2.000000
75%,2.120000e+05,1001.000000,4.000000,64.000000,6.000000,7.000000,16.000000,2.000000
max,2.210000e+06,100001.000000,32.000000,1616.000000,7.200000,24.000000,32.000000,2.000000


# DETECTING AND REMOVING OUTLIERS
An outlier is a data point that differs significantly from other observations we can easily look at the max of the numeric values 


1. From the summary statistic above we can clearly see that the max prise is too high  so wan can investigate
  After the investigation below to see wich phone is sold at 22100000 we see that its  'Galaxy S6 Edge - 32Go HDD - RAM 3Go' which is not possible so we drop it 
 2. we can find outliers in the rom section since no phone till now sell on jumia.cm has a rom of 1616GO so we can check and fix since we can clearly see that the real value is on the item name(the difficulty to get it from the name is due to the inconsistency of the item name )

In [11]:
print(smartphones.loc[smartphones['price']>2000000])
print('--------------------------- outliers------------------------------------------------')
''' Checking the outliers in the rom section'''
print(smartphones.loc[smartphones['rom']>258])

'''We drop the item that is supposed to cost 2210000'''
smartphones.drop(451, 0, inplace=True)

''' After detecting it we can fix it'''
smartphones.at[229, 'rom']=8
smartphones.at[229, 'ram']=1
smartphones.at[339, 'rom']=126
smartphones.at[339, 'ram']=4
smartphones.at[611, 'rom']=32
smartphones.at[613, 'rom']=8

                                                  name   seller      price  \
451  Galaxy S6 Edge - 32Go HDD - RAM 3Go - 5,7" - O...  Samsung  2210000.0   

     number-of-items-sold  ram   rom  screen-size color  front-camera  \
451                  49.0  3.0  64.0          5.7    or           8.0   

     rear-camera  number_of_sims                      date       os  
451         12.0               1  Sun May 26 21:33:11 2019  android  
--------------------------- outliers------------------------------------------------
                                                  name   seller     price  \
229  Galaxy J2 Core Dual Sim - 8Go HDD + 1Go RAM - ...  Samsung   56800.0   
339  Redmi Note 7 Dual SIM - 128Go - 4Go Ram HDD - ...    Xaomi  200500.0   
611                        iPhone 7 Plus 32Go HDD - Or    Apple  513600.0   
613            A2 XL Dual SIM 8Go HDD - Noir Et Argent  Alcatel   45000.0   

     number-of-items-sold  ram     rom  screen-size           color  \
229           

/home/devherve/anaconda3/lib/python3.7/site-packages/pandas/core/frame.py:3697: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  errors=errors)


## Our dataset is clean to a certain extend so we can analyse it now

### let's create a new column computing the price of all items sold

In [21]:
''' Having rename the dataset we are working with as smartphones
 Here we compute the '''
smartphones['total-price']=smartphones['price'] *  smartphones[ 'number-of-items-sold']
smartphones.head()

/home/devherve/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


,name,seller,price,number-of-items-sold,ram,rom,screen-size,color,front-camera,rear-camera,number_of_sims,date,os,total-price
0,Rise 31 Dual SIM 8Go HDD - Bleu + 1 Carte SIM ...,Orange,24900.0,1001.0,1.0,8.0,4.0,bleu,13.0,21.0,2,Sun May 26 21:18:11 2019,android,24924900.0
1,"Kodak Ektra - 5"" - 32 Go HDD - 3 Go RAM - Noir",KODAK,49900.0,1001.0,3.0,32.0,5.0,noir,8.0,13.0,1,Sun May 26 21:18:13 2019,android,49949900.0
2,Spark 2 Dual SIM 16 Go HDD + 1 Go RAM - Rouge ...,Tecno,85000.0,1001.0,1.0,16.0,6.0,rouge,8.0,13.0,2,Sun May 26 21:18:14 2019,android,85085000.0
3,Spark 2 Dual SIM 16 Go HDD - Noir - 12 Mois,Tecno,75300.0,101.0,1.0,16.0,5.0,noir,0.0,8.0,2,Sun May 26 21:18:16 2019,android,7605300.0
4,Rise 53 Dual SIM 8Go HDD - Noir - 12 Mois + Fo...,Orange,34900.0,1001.0,1.0,8.0,3.5,noir,13.0,8.0,2,Sun May 26 21:18:17 2019,android,34934900.0


# GETTING SOME INSIGHT FROM OUR DATA

1. Bellow we clearly see most sold phones on the platform and their prices 
    We have 7 phones that the number of sales is above 100000
    
2. Samsung  has 296 varieties of equipments on the platform ,Apple 104, Tecno 60, Huawei 44, ITEL 44, Xiaomi          37,Infinix 36, Lenovo 17, X-tigi 16,LG 13 and the rest bellow 10

3. for the total numbers of items sold, Samsung is leading with 461970.0 items sold, Huawei 236752.0, Xiaomi          209343.0, Infinix 156974.0, ITEL  133346.0 and LG 101717.0  and the rest below 100000

4. from the analysis we clearly see that sumsung phones have been sold for 81947570000xaf(140736756.72usd), apple      44062290000xaf( 75672576.85usd) etc...


## as the result of all this, we can see that jumia.cm has sold more than at at total of 236153558184xaf( 405570120.83 usd)

In [67]:
'''Most sale smartphones on jumia at the date of Sun May 26 21:24:44 2019 '''
most_sell = smartphones['number-of-items-sold'].max()
print('the maximum number of items sold is  {0}'.format(most_sell))
'''Lets get the most sold items '''
most_sold_items= smartphones.loc[smartphones['number-of-items-sold']==most_sell]
print(most_sold_items)

#print(smartphones.loc[smartphones['number-of-items-sold'].idxmax()])

''' making a good print of it'''
print('-------------Below most sold items and their sellers-------------------------------')
print(most_sold_items[['name', 'seller', 'price', 'number-of-items-sold']])

'''numbers of items own by a specific seller'''
sellers = smartphones['seller'].astype('category').value_counts()
print('-------------------numbers of items own by a specific seller---------------------------------')
print(sellers)

the maximum number of items sold is  100001.0
                                                  name   seller     price  \
198  Mi A2 Dual SIM - 64Go HDD - 4Go RAM - 20+12MP ...   Xiaomi  137900.0   
208     Note 5 Stylus Dual SIM 64 Go HDD - Or  Et Noir  Infinix  140000.0   
222  G6 64Go HDD - 4Go RAM - Or - 12 Mois + Pochett...       LG  124900.0   
258  A14 - Dual Sim - 8Go HDD - 512 Mo RAM - 4" - N...     ITEL   28000.0   
285                             Y3 2017 8Go HDD - Gris   Huawei   64900.0   
288    Y3 2017 8Go HDD - Or + 1 Modem Airbox 3G Offert   Huawei   84900.0   
289  Galaxy Grand Prime Dual Sim 8 Go HDD - Argent ...  Samsung   67900.0   

     number-of-items-sold  ram   rom  screen-size       color  front-camera  \
198              100001.0  2.0  64.0          6.2        noir           2.0   
208              100001.0  1.0  16.0          6.4  or et noir           1.2   
222              100001.0  1.0  16.0          5.0          or           5.0   
258              1000

In [68]:

''' best seller with highest number of items sold'''
best = smartphones['number-of-items-sold'].groupby(smartphones['seller']).sum()

print('-----------------------number of equipments sold by each seller---------------------')
print(best.sort_values(ascending=False))



-----------------------number of equipments sold by each seller---------------------
seller
Samsung      461970.0
Huawei       236752.0
Xiaomi       209343.0
Infinix      156974.0
ITEL         133346.0
LG           101717.0
Apple         98210.0
Lenovo        95213.0
Tecno         71902.0
Alcatel       15513.0
ZTE           13255.0
Nokia         11551.0
Xaomi         11199.0
COOLPAD       11103.0
Noa           11002.0
X-tigi         7662.0
Oking          2653.0
HTC            2354.0
Orange         2152.0
MOTOROLA       1109.0
Microsoft      1102.0
Fero           1050.0
TCL            1050.0
Blu            1050.0
Oppo           1001.0
Maku           1001.0
KODAK          1001.0
VSUN           1001.0
X TOUCH         319.0
TORQUE          299.0
Gionee          299.0
SIMI            297.0
UMIDIGI         202.0
Flash           108.0
mione            99.0
PHICOMM          99.0
WIKO             99.0
ASUS             99.0
Xtouch           98.0
ULEFONE           9.0
Name: number-of-items-sold, 

In [72]:
''' amount generated by each marque on the websites'''
totalCashByBrand= smartphones['total-price'].groupby(smartphones['seller']).sum()
print(totalCashByBrand.sort_values(ascending=False))

seller
Samsung      8.194757e+10
Apple        4.406229e+10
Xiaomi       2.766623e+10
Huawei       2.381001e+10
Infinix      1.958523e+10
LG           1.284149e+10
Lenovo       8.974123e+09
Tecno        7.684863e+09
ITEL         4.080605e+09
Xaomi        1.085976e+09
Alcatel      9.841980e+08
Oppo         5.063058e+08
Nokia        4.997295e+08
COOLPAD      4.730127e+08
Noa          3.850700e+08
ZTE          3.823093e+08
X-tigi       3.162114e+08
HTC          1.121845e+08
Oking        1.096751e+08
TCL          9.852500e+07
Orange       6.559980e+07
VSUN         5.445440e+07
Fero         5.111400e+07
Microsoft    5.077730e+07
KODAK        4.994990e+07
MOTOROLA     4.570090e+07
X TOUCH      4.317900e+07
Gionee       3.719700e+07
Blu          3.209640e+07
Maku         3.203200e+07
UMIDIGI      2.979500e+07
TORQUE       1.106300e+07
SIMI         1.042470e+07
ASUS         9.454500e+06
Xtouch       6.125000e+06
PHICOMM      4.851000e+06
WIKO         4.742100e+06
Flash        4.548600e+06
mione

In [74]:
'''cash generated by the items in our dataset'''
total = smartphones['total-price'].sum()
totalSold=smartphones['number-of-items-sold'].sum()
print('---------------------the cash generated in total is -------------------------')
print('Jumia.cm has generated {0}xaf by selling {1} phones we have in our dataset'.format(total, totalSold))

---------------------the cash generated in total is -------------------------
Jumia.cm has generated 236153558184.0xaf by selling phones we have in our dataset
